## This colab contains the code to pretrain an LM model on OGBN Products and dump the resultant embeddings

In [1]:
!pip install sentence-transformers
!pip install ogb
from sentence_transformers import SentenceTransformer, models
from ogb.nodeproppred import NodePropPredDataset
import pandas as pd
import numpy as np
import pickle
import math
from tqdm import tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.7 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=141c16ae709d884cbe372a496d05517a7f4796b4e468161cc9ea88b1442de122
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7026 sha256=d3c7b2f172e45931a6a0dc5657830f41d3bb979a25dd08c2d49cfe0e65c4662c
  Stored in directory: /root/.cache/pip/wheels/3d/fe/b0/27a9892da57472e538c7452a721a9cf463cc03cf7379889266
Successfully built littleutils


## Link the Colab to your Google Drive

We use Google Drive to load the datasets and dump the trained model.

The dataset is available at https://drive.google.com/drive/folders/10xPY3Bv6ugkJX7pAEHYPwXex234b1uWg?usp=sharing. Please create a copy of this folder in your google drive and update the DATA_ROOT to point to this

In [2]:
MODEL = "roberta-large"
DATASET = "ogbn-products"
OUTPUT_FILE = ""
MODE = 'all'
DATA_ROOT = "/content/drive/Shareddrives/CS224W Project/"

In [3]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


## Load Node-Paper Id Mappings

In [8]:
nodeidx2productid = pd.read_csv(DATA_ROOT+'ogbn-products_subset.csv')
nodeidx2productid.head()

,uid,nid,title,content
0,B00JXMX35K,1324660,Aimee Gowns Original Bra-less Nursing Gown (X-...,The one that started it all! The nightgown's m...
1,B000XQHYFU,1670226,Casio DR-270TM 2-Color Professional Desktop Pr...,Heavy duty printing
2,B00B67RO9Q,1780321,NaN,None
3,B005JN9CKM,2447868,Muay Thai Shorts-Black,"for Size Chart, please see attached photos. Im..."
4,B00CU9YBVS,1940019,Smatree&reg; Chest Belt/Strap Harness Mount+ A...,1. Smatree chest strap is fully adjustable to ...


## Load Paper Mappings

In [5]:
titleabs = pd.read_csv(DATA_ROOT+'titleabs.tsv', sep='\t')
titleabs.head()

,paperid,title,abstract
0,200971.0,ontology as a source for rule generation,This paper discloses the potential of OWL (Web...
1,549074.0,a novel methodology for thermal analysis a 3 d...,The semiconductor industry is reaching a fasci...
2,630234.0,spreadsheets on the move an evaluation of mobi...,The power of mobile devices has increased dram...
3,803423.0,multi view metric learning for multi view vide...,Traditional methods on video summarization are...
4,1102481.0,big data analytics in future internet of things,Current research on Internet of Things (IoT) m...


In [6]:
titleabs.keys()

Index(['paperid', 'title', 'abstract'], dtype='object')

# Make reverse index for text df


In [9]:
reverse_index= {}

productids = nodeidx2productid["nid"].tolist()
for idx, productid in enumerate(productids):
    if (not math.isnan(productid)):
        reverse_index[int(productid)] = idx

## Dataset Creation


In [10]:

mapped_text = []

for idx, product_id in tqdm(enumerate(productids)):
    title = nodeidx2productid.iloc[idx]['title']
    content = nodeidx2productid.iloc[idx]['content']
    if (MODE == 'title'):
        mapped_text.append("Title: " + str(title))
    elif (MODE == 'abstract'):
        mapped_text.append(" Abstract: " + str(content))
    else:
        mapped_text.append("Title: " + str(title) + " Abstract: " + str(content))

54025it [00:12, 4188.46it/s]


## Initialize sentence embedding model


In [ ]:

word_embedding_model = models.Transformer(MODEL)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), pooling_mode = 'mean')
emb_model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

## Get NumPy embeddings


In [ ]:

embeddings = emb_model.encode(sentences = mapped_text,
                             batch_size = 32,
                             show_progress_bar = True,
                             convert_to_numpy = True,
                             normalize_embeddings = True)

# Dump Embeddings


In [ ]:

with open(OUTPUT_FILE, 'wb') as f:
    pickle.dump(embeddings, f)